# Lecture 4 - Noise and Benchmarking

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import qiskit as qs
from qiskit_ibm_runtime import QiskitRuntimeService

from qiskit.transpiler import generate_preset_pass_manager
from qiskit.visualization import timeline
import qiskit_experiments

In [ ]:
# Run this to install qiskit experiments for T1, T2, and other kinds of benchmarks
!pip install qiskit-experiments[extras]

In [ ]:
# Setup the backend and transpiler

service = QiskitRuntimeService(name='rpi-quantum')
backend = service.backend('ibm_rensselaer')

pm = generate_preset_pass_manager(backend=backend,
                                  optimization_level=1,
                                  scheduling_method='asap')

## Density Matrices

In [ ]:
zero_state = np.array([1.0+0j, 0.0+0j])
one_state =  np.array([0.0+0j, 1.0+0j])

zero_state

In [ ]:
P0 = np.outer(zero_state, zero_state)
P1 = np.outer(one_state, one_state)

In [ ]:
P0

In [ ]:
X = np.array([[0,1],[1,0]])
Y = np.array([[0,0.0-1.0j],[0.0+1.0j,0.0]])
Z = np.array([[1,0],[0,-1]])
H = 1/np.sqrt(2) * np.array([[1, 1],[1, -1]])

In [ ]:
plus_state = H @ zero_state
plus_state

In [ ]:
rho_H = np.outer(plus_state, plus_state)
rho_H

## Measuring $T_1$ (The Hard Way)

$T_1$ tells us the time constant with which a qubit will decay from the $|1\rangle$ state to the $|0\rangle$ state. If we want to measure this value, we need to prepare the $|1\rangle$ state many, many times, and then compute the average probability of the qubit being in that state after various waiting times.

In [ ]:
n_waits = 11
wait_times = np.linspace(0.0, 200.0, n_waits)*1e-6
wait_times

Now, we create a circuit for each delay time (we could also use `Parameter`s).

In [ ]:
t1_circuits = []

# Create a circuit for each delay time
for wait_time in wait_times:
    t1_circuit = qs.QuantumCircuit(1)
    
    # Use an X-gate to flip the qubit into the 1 state.
    t1_circuit.x(0)
    
    t1_circuit.delay(duration=wait_time, unit='s')

    t1_circuits.append(t1_circuit)

observable = qs.quantum_info.SparsePauliOp(["I", "Z"], [0.5, -0.5])

In [ ]:
isa_t1_circuits = pm.run(t1_circuits)
isa_observables = [observable.apply_layout(circuit.layout) for circuit in isa_t1_circuits]

In [ ]:
estimator = ibm.EstimatorV2(mode=backend)

pubs = []
for i in range(len(isa_t1_circuits)):
    pub = (isa_t1_circuits[i], [isa_observables[i]])
    pubs.append(pub)

job = estimator.run(pubs=pubs)

In [ ]:
job.status()

In [ ]:
result = job.result()

In [ ]:
evs = []
for item in result:
    ev = item.data.evs[0]

    evs.append(ev)

In [ ]:
plt.plot(wait_times, evs, marker="o")

## Measuring $T_1$ (A Slightly Easier Way)

Qiskit Experiments gives us an easier way to measure values like $T_1$, $T_2$ and many others. Instead of having to write a new experiment loop for everything, we can import these predefined experiemnts from a library. It's also much easier for us to pick the qubits we want to target.

In [ ]:
t1_exps = []
qubits = [28]

for qubit in qubits:
    
    # Create the experiment for the qubit
    exp = T1(physical_qubits=(qubit,),
             delays=wait_times,
             backend=backend,)
    
    # Add it to our list of experiments
    t1_exps.append(exp)

parallel_t1_exp = ParallelExperiment(t1_exps, backend=backend,)

In [ ]:
parallel_t1_exp.set_transpile_options(scheduling_method='asap',
                                      target=backend.target)

t1_data = parallel_t1_exp.run(backend=backend)

In [ ]:
t1_data.figure(0)

## Measuring $T_2$
We can do the same thing for $T_2$. The Hahn Echo Sequence is built into Qiskit Experiments.

In [ ]:
t2_delays = np.linspace(0.0, 100.0, 26)*1e-6
t2_delays

In [ ]:
t2_exps = []
qubits = [28]

for qubit in qubits:
    
    # Create the experiment for the qubit
    exp = T2Hahn(physical_qubits=(qubit,),
                 delays=t2_delays,
                 backend=backend,)
    
    # Add it to our list of experiments
    t2_exps.append(exp)

parallel_t2_exp = ParallelExperiment(t2_exps, backend=backend,)

In [ ]:
parallel_t2_exp.set_transpile_options(scheduling_method='asap', target=backend.target)
t2_data = parallel_t2_exp.run(backend=backend)

In [ ]:
t2_data.figure(0)

## Measuring $T_2^*$

The Ramsey measurement sequence is also built in to the experiment library!

In [ ]:
t2star_exps = []
qubits = [28]

for qubit in qubits:
    
    # Create the experiment for the qubit
    exp = T2Ramsey(physical_qubits=(qubit,),
                   delays=t2_delays,
                   backend=backend,
                   osc_freq=1e6)
    
    # Add it to our list of experiments
    t2star_exps.append(exp)

parallel_t2star_exp = ParallelExperiment(t2_exps, backend=backend,)

In [ ]:
parallel_t2star_exp.set_transpile_options(scheduling_method='asap', target=backend.target)
t2star_data = parallel_t2star_exp.run(backend=backend)

In [ ]:
t2star_data.figure(0)

## Determining the Gate Time

The next step in making a noise model is figure out how long it takes to run operations. Ideally, these should be as short as possible

In [ ]:
circ = qs.QuantumCircuit(1)
circ.h(0)

isa_circ = pm.run(circ)

gate_time = isa_circ.duration*backend.dt
print(f"Gate time for Hadamard gate: {gate_time/1e-9} ns")

timeline.draw(isa_circ, idle_wires=False, show_delays=True)

From the above, we can see that the gate time is about 60 nanoseconds.

## Noise Modeling Example
Here we'll show an example of how we can use measured $T_1$ and $T_2$ values, along with the gate time $T_g$ to create a simple noise model.

In [ ]:
T1 = t1_data.analysis_results()[1].value.nominal_value
T2 = t2_data.analysis_results()[1].value.nominal_value
Tg = gate_time

In [ ]:
t1_damping = 1 - np.exp(-Tg/T1)
t2_damping = 1 - np.exp(-Tg/T2)

Now, we will construct the Kraus operators for the noise model.

In [ ]:
E0_amp = np.array([[1,0],[0, np.sqrt(1-t1_damping)]])
E1_amp = np.array([[0,np.sqrt(t1_damping)],[0, 0]])

In [ ]:
E0_phase = np.array([[1,0],[0, np.sqrt(1-t2_damping)]])
E1_phase = np.array([[0,0],[0, np.sqrt(t2_damping)]])

Let's test it! First, we'll create a density matrix of the $|0\rangle$ state:

In [ ]:
zero_state = np.array([1.0+0j, 0.0+0j])
hadamard_gate = 1/np.sqrt(2) * np.array([[1, 1],[1, -1]])

rho_1 = np.outer(zero_state, zero_state)

Next, we apply a Hadamard gate to it. Remember that in the density matrix formalism, we apply operators like:

$$ \rho^\prime = U\rho U^\dagger$$

Which gives us:

$$ \rho_2 = H\rho_1H^\dagger = \frac{1}{\sqrt{2}}\begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix}\rho  \frac{1}{\sqrt{2}} \begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix}$$
$$ \rho_2 = \begin{pmatrix} \frac{1}{2} & \frac{1}{2} \\ \frac{1}{2} & \frac{1}{2} \end{pmatrix} $$

In [ ]:
rho2 = (hadamard_gate @ rho @ hadamard_gate.conj().T)
rho2

Next, we apply the amplitude damping operator:

In [ ]:
rho3 = E0_amp @ rho2 @ E0_amp.conj().T + E1_amp @ rho2 @ E1_amp.conj().T

Then, we apply the phase damping operator:

In [ ]:
rho4 = E0_phase @ rho3 @ E0_phase.conj().T + E1_phase @ rho3 @ E1_phase.conj().T

In [ ]:
rho4

Notice how the value in the top right element increased, and the other elements all decreased. This noise is causing the measurement probability of the qubit to drift back to the $|0\rangle$ state. 

What do you think will happen if we increase the gate time?